# Language Generation Evaluation

### Analysis of the Europarl Dataset

In [226]:
import nltk
import csv
import random
import numpy as np
import Levenshtein
from random_word import RandomWords
from collections import Counter
from nltk.corpus import wordnet
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import pandas
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oskar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [227]:
# # Load the dataset
# def load_data(danish_file, english_file):
#     with open(danish_file, "r", encoding="utf-8") as f_da, open(english_file, "r", encoding="utf-8") as f_en:
#         danish_sentences = [line.strip() for line in f_da.readlines()]
#         english_sentences = [line.strip() for line in f_en.readlines()]
#     return danish_sentences, english_sentences

# danish_sentences, english_sentences = load_data("../da.da", "../en.en")

# # Tokenization
# danish_tokens = [word_tokenize(sent.lower()) for sent in danish_sentences]
# english_tokens = [word_tokenize(sent.lower()) for sent in english_sentences]

# # Flatten lists for word statistics
# danish_words = [word for sent in danish_tokens for word in sent]
# english_words = [word for sent in english_tokens for word in sent]



In [228]:
# num_sentences_da = len(danish_sentences)
# num_sentences_en = len(english_sentences)

# num_words_da = len(danish_words)
# num_words_en = len(english_words)

# unique_words_da = len(set(danish_words))
# unique_words_en = len(set(english_words))

# ttr_da = unique_words_da / num_words_da * 100
# ttr_en = unique_words_en / num_words_en * 100

# avg_length_da = np.mean([len(sent) for sent in danish_tokens])
# avg_length_en = np.mean([len(sent) for sent in english_tokens])

# std_length_da = np.std([len(sent) for sent in danish_tokens])
# std_length_en = np.std([len(sent) for sent in english_tokens])

# min_length_da = np.min([len(sent) for sent in danish_tokens])
# min_length_en = np.min([len(sent) for sent in english_tokens])

# max_length_da = np.max([len(sent) for sent in danish_tokens])
# max_length_en = np.max([len(sent) for sent in english_tokens])

# sentence_length_ratio = avg_length_da / avg_length_en 

# print(f"Total Sentences: Danish = {num_sentences_da}, English = {num_sentences_en}")
# print(f"Total Words: Danish = {num_words_da}, English = {num_words_en}")
# print(f"Unique Words: Danish = {unique_words_da}, English = {unique_words_en}")
# print(f"Type-Token Ratio: Danish = {ttr_da:.2f}%, English = {ttr_en:.2f}%")
# print(f"Avg. Sentence Length: Danish = {avg_length_da:.2f}, English = {avg_length_en:.2f}")
# print(f"Std Dev Sentence Length: Danish = {std_length_da:.2f}, English = {std_length_en:.2f}")
# print(f"Sentence Length Ratio (DA/EN): {sentence_length_ratio:.2f}")
# print(f"Min sentence length (DA/EN): Danish = {min_length_da}, English = {min_length_en}")
# print(f"Max sentence length (DA/EN): Danish = {max_length_da}, English = {max_length_en}")

# Create dataframes

In [229]:
# name = "English"

# with open("../en.en", "r", encoding="utf-8") as f:
#     for i in range(200):
#         lines = [next(f).strip() for _ in range(10000)]
#         df = pandas.DataFrame({"Text": lines})
#         output_file = f"dataframes/en/{name}{i+1}.csv"
#         df.to_csv(output_file, index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)

# name = "Danish"

# with open("../da.da", "r", encoding="utf-8") as f:
#     for i in range(200):
#         lines = [next(f).strip() for _ in range(10000)]
#         df = pandas.DataFrame({"Text": lines})
#         output_file = f"dataframes/da/{name}{i+1}.csv"
#         df.to_csv(output_file, index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)


# danishData = pandas.read_fwf("test.txt", header=None, names=["test_sentences"])
# englishData = pandas.read_fwf("test.txt", header=None, names=["test_sentences"])

### Load dependencies (First, look at README)

In [230]:
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize
from nltk.translate import meteor
import nltk
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\oskar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\oskar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Define functions

In [ ]:
rouge = Rouge()
r = RandomWords()

def calculate_rouge(candidate, reference):
    scores = rouge.get_scores(candidate, reference)
    return scores

def calculate_bleu(candidate, reference):
    reference_p = [word_tokenize(reference)]
    candidate_p = word_tokenize(candidate)
    score = sentence_bleu(reference_p, candidate_p)
    return score

def calculate_meteor(candidate, reference):
  reference = word_tokenize(reference)
  candidate = word_tokenize(candidate)
  meteor_score = round(meteor([candidate],reference), 4)
  return meteor_score

### Perform Rouge test

In [232]:
#print(calculate_rouge(chatGptTrans, proTrans))

### Perform BLEU test

In [233]:
#print(calculate_bleu(chatGptTrans, proTrans))

### Perform METEOR test

In [234]:
#print(calculate_meteor(chatGptTrans, proTrans))

# Replace with synonym

In [426]:
def findSynonym(w):
    res = []
    for x in wordnet.synsets(w):
        for y in x.lemma_names():
            res.append(y.replace("_", " "))
    if len(res) > 0:
        word = random.choice(list(set(res)))
        return [not word==w, word]
    else: return [False, ""] 

def replaceWordWithSynonym(sen):
    toks = word_tokenize(sen.lower())
    indx = random.randrange(len(toks))
    syn = findSynonym(toks[indx])
    if syn[0]:
        res = toks
        res[indx] = syn[1]
        return True, TreebankWordDetokenizer().detokenize(res)
    else: return False, TreebankWordDetokenizer().detokenize(toks)

print(replaceWordWithSynonym("Hello my name is Oskar and I am 28 years old"))

(True, 'hello my name personify oskar and i am 28 years old')


# Replace with random word

In [432]:
def replaceWordWithSynonym(sen):
    randomWord = r.get_random_word()
    toks = word_tokenize(sen.lower())
    indx = random.randrange(len(toks))
    toks[indx] = randomWord
    return TreebankWordDetokenizer().detokenize(toks)

print(replaceWordWithSynonym("Hello what is going on"))

hello cogitabundity is going on


# Swap Words

In [435]:
def swapXWords(sen):
    words = sen.count(" ") + 1
    toSwap = random.randint(0, words-2)
    wordList = sen.split()
    tmp = wordList[toSwap]
    wordList[toSwap] = wordList[toSwap+1]
    wordList[toSwap+1] = tmp
    res = ""
    for x in wordList:
        res += x + " "
    return res.lower()

print(swapXWords("Hello what are you doing right now"))

hello what are you right doing now 


# Remove Word

In [439]:
def removeWord(sen):
    toks = word_tokenize(sen.lower())
    indx = random.randrange(len(toks))
    del toks[indx]
    return TreebankWordDetokenizer().detokenize(toks)

print(removeWord("Hello what are you doing right now"))

hello what are doing right now


# Add random word

In [452]:
def addRandomWord(sen):
    toks = word_tokenize(sen.lower())
    indx = random.randrange(len(toks))
    toks.insert(indx, r.get_random_word())
    return TreebankWordDetokenizer().detokenize(toks)

print(addRandomWord("Hello, what are you doing?"))

hello, what are ulotrichi you doing?


# Duplicate word

In [455]:
def duplicateWord(sen):
    toks = word_tokenize(sen.lower())
    indx = random.randrange(len(toks))
    word = toks[indx]
    toks.insert(indx, word)
    return TreebankWordDetokenizer().detokenize(toks)

print(duplicateWord("Hello, what are you doing?"))

hello, what are you doing doing?


# Character replacement

In [464]:
def replaceCharacter(sen):
    lst = list(sen)
    indx = random.randrange(len(lst))
    lst[indx] = random.choice("abcdefghijklmnopqrstuvwxyz")
    return "".join(lst)

print(replaceCharacter("Hello, what are you doing?"))

Hello, what are you koing?


# Character Duplication

In [466]:
def duplicateCharacter(sen):
    lst = list(sen)
    indx = random.randrange(len(lst))
    lst.insert(indx, lst[indx])
    return "".join(lst)

print(duplicateCharacter("Hello, what are you doing?"))

Hello, what are you doiing?


# Character Removal

In [382]:
def removeCharacter(sen):
    lst = list(sen)
    indx = random.randrange(len(lst))
    del lst[indx]
    return "".join(lst)

print(removeCharacter("Hello, what are you doing?"))

Hello, what are you doig?


# Character Swapping

In [398]:
def swapCharacters(sen):
    lst = list(sen)
    indx = random.randrange(len(lst)-1)
    temp = lst[indx]
    lst[indx] = lst[indx+1]
    lst[indx+1] = temp
    return "".join(lst)

print(swapCharacters("Hello, what are you doing?"))

Hello, what are oyu doing?
